In [3]:
import os

In [4]:
from keras.preprocessing import image

In [5]:
#!pip install opencv-python

In [6]:
import cv2

In [7]:
categories = ['with_mask','without_mask']

In [8]:
data = []
for category in categories:
    path = os.path.join('train',category)
    
    label = categories.index(category)
    
    for file in os.listdir(path):
        
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))
        
        data.append([img,label])

In [9]:
len(data)

1600

In [10]:
import random

In [11]:
random.shuffle(data)

In [12]:
x = []
y = []

for features,label in data:
    x.append(features)
    y.append(label)

In [13]:
len(x)

1600

In [14]:
len(y)

1600

In [15]:
import numpy as np

In [16]:
x = np.array(x)
y = np.array(y)

In [17]:
x.shape

(1600, 224, 224, 3)

In [18]:
y.shape

(1600,)

In [19]:
x=x/255

In [20]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [21]:
x_train.shape

(1280, 224, 224, 3)

In [22]:
x_test.shape

(320, 224, 224, 3)

In [23]:
from keras.applications.vgg16 import VGG16

In [24]:
vgg = VGG16()

In [25]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [26]:
from keras import Sequential

In [27]:
model = Sequential()

In [28]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [30]:
for layer in model.layers:
    layer.trainable=False

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [32]:
from keras.layers import Dense

In [33]:
model.add(Dense(1,activation='sigmoid'))

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [35]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [34]:
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
 1/40 [..............................] - ETA: 4:13 - loss: 0.8317 - accuracy: 0.5000

KeyboardInterrupt: 

In [36]:
cap = cv2.VideoCapture(0)

In [37]:
def detect_face_mask(img):
    
    y_pred = model.predict(img.reshape(1,224,224,3))
    
    return y_pred[0][0]



In [38]:
sample1 = cv2.imread('sample1.jpg')
sample1 = cv2.resize(sample1,(224,224))

In [39]:
detect_face_mask(sample1)

1/1 [==============================] - 1s 1s/step


0.6938645

In [40]:
def draw_label(img,text,pos,bg_color):
    
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] - text_size[0][1] - 2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [41]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [42]:
def detect_face(img):
    
    coods = haar.detectMultiScale(img)
    
    return (coods)

In [ ]:
while True:
    
    ret, frame = cap.read()
    
    # call the detection method
    img = cv2.resize(frame,(224,224))
    
    y_pred = detect_face_mask(img)
    
    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))
    
    for x,y,w,h in coods:
        
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
     
    
    if y_pred == 0:
        draw_label(frame,"No Mask", (30,30),(0,0,255))
    else:
        draw_label(frame,"Mask", (30,30),(0,255,0))
        
        
    
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break
                
cv2.destroyAllWindows()      

1/1 [==============================] - 0s 221ms/step
